# Chapter 9. Optimizing the ESN's Internal Connection

In this chapter, you will learn about RC schemes that adjust the internal connections of ESNs through online learning.

**Note:** The code in this chapter does not work on Google Colaboratory.
Please refer to the README to set up a local environment and run the code there.

## Introduction

In standard ESN setups, the internal connections of the ESN are fixed.
The internal weight, however, can be considered a type of closed loop.
For example, consider the following ESN:

$$
\begin{align*}
x[k+1] &= \tanh\left(\rho W^\mathrm{rec} x[k] + W^\mathrm{in} u[k+1]+ W^\mathrm{feed} y[k] \right)\\
y[k] &= W^\mathrm{out}x[k]
,\end{align*}
$$

which can be rewritten as the following equation:

$$
\begin{align*}
x[k+1] &= \tanh\left(\rho W^\mathrm{rec} x[k] + W^\mathrm{in} u[k+1]+ W^\mathrm{feed}W^\mathrm{out}x[k] \right)\\
&= \tanh\left(\left(\rho W^\mathrm{rec} + W^\mathrm{feed}W^\mathrm{out}\right) x[k] + W^\mathrm{in} u[k+1] \right)
.\end{align*}
$$

In other words, the internal connection can be viewed as $\rho W^\mathrm{rec} + W^\mathrm{feed}W^\mathrm{out}$.
Conversely, the pre-synaptic connection $W^\mathrm{rec}_{i~\cdot}$ and the post-synaptic connection $W^\mathrm{rec}_{\cdot~i}$ of the $i$-th node can be externalized as $W^\mathrm{out}$ and $W^\mathrm{feed}$ in a feedback loop, respectively.
This means that internal connections can be adjusted using the closed-loop learning methods introduced in the previous chapter.
A new question then arises: how to specify the target time-series data for the internal connection.
Both **innate training**<sup>[1]</sup> and **full-FORCE**<sup>[2]</sup> generate a target time series using a different ESN and train the internal connection with FORCE learning<sup>[3]</sup>.

### Innate training

**Innate training**<sup>[1]</sup> is an online learning method that adjusts the internal connections of ESNs, proposed by R. Laje et al.
Innate training uses a chaotic ESN, and another ESN's internal connections are tuned to reproducibly generate the chaotic dynamics over a certain period.
This target trajectory is called an **innate trajectory**.

The learning process consists of pre-training and post-training.
In pre-training, the internal connections are adjusted using FORCE learning<sup>[3]</sup>.
Post-training is performed after pre-training, similar to standard RC.
In other words, the readout layer is trained to output a specific trajectory in an open-loop system.
The cost function $C(W^\mathrm{rec})$ of the pre-training is expressed as follows:

$$
\begin{align*}
C(W^\mathrm{rec}):= \sum_{k \in T} \|x^\mathrm{target}[k] - x[k]\|^2
,\end{align*}
$$

where $x^\mathrm{target}[k]$ is an innate trajectory, and $T$ is a time range.
In innate training, a time series for a given input with an initial value $x^\mathrm{target}[t_0]$ is often used as a target.
Interestingly, the chaoticity is not completely suppressed even after pre-training.
That is, even though the system is globally chaotic, the ESP locally recovers, and high-dimensional complex trajectories are reproducibly generated for a certain period.

In addition, as will be shown in the demonstration, the high-dimensional chaotic trajectory has high expressiveness.
This high-dimensional chaos can embed not only local patterns but also global switching rules between them<sup>[4]</sup> (see [the code provided by the paper's authors](https://github.com/katsuma-inoue/designing_chaotic_itinerancy_demo)).

### Full-FORCE

**Full FORCE**<sup>[2]</sup>, proposed by B. DePasquale et al., is another online learning method that adjusts the internal connections of an ESN.
Unlike innate training, it does not use a chaotic ESN and its trajectory as the target time series.
Instead, it uses the trajectory of a non-chaotic ESN ($\rho<1$) with an additional input $u^\mathrm{embed}[k]$, expressed by the following equation:

$$
\begin{align*}
x^\mathrm{target}[k+1] &= (1-a)x^\mathrm{target}[k] + a\tanh\left(\rho W^\mathrm{rec} x^\mathrm{target}[k] + W^\mathrm{in} u^\mathrm{in}[k]+ W^\mathrm{embed} u^\mathrm{embed}[k] \right)
,\end{align*}
$$

where $u^\mathrm{embed}[k]$ is an additional input to be embedded and is a function of $u^\mathrm{in}[k]$.
Full-FORCE aims to design an ESN that replicates the dynamics **without** this $u^\mathrm{embed}[k]$.
The cost function $C(W^\mathrm{rec})$ is expressed as follows:

$$
\begin{align*}
C(W^\mathrm{rec}):= \sum_{k \in T} \|x^\mathrm{target}[k] + W^\mathrm{embed}u^\mathrm{embed}[k] - x[k]\|^2
,\end{align*}
$$

Full-FORCE learning also involves post-training, where an additional readout layer is adjusted.
A key feature of full-FORCE is that embedding $W^\mathrm{embed} u^\mathrm{embed}[k]$ allows for designing more complex time series compared to FORCE learning alone.
In this way, full-FORCE improves the design capability of time series by adjusting the internal connections.

## Exercises and demonstrations

Execute the following cell to import the classes that we implemented previously, as well as the basic libraries.
You can check the internal implementations by uncommenting the lines after `import inspect` or by using `...?? / ??...`.

In [ ]:
import json
import os
import sys
from collections import defaultdict

import joblib
import numpy as np
from IPython.display import clear_output, display
from ipywidgets import Output

if "google.colab" in sys.modules:
    from google.colab import drive  # type: ignore

    if False:  # Set to True if you want to use Google Drive and save your work there.
        drive.mount("/content/gdrive")
        %cd /content/gdrive/My Drive/rc-bootcamp/
        # NOTE: Change it to your own path if you put the zip file elsewhere.
        # e.g., %cd /content/gdrive/My Drive/[PATH_TO_EXTRACT]/rc-bootcamp/
    else:
        pass
        %cd /content/
        !git clone --branch en https://github.com/rc-bootcamp/rc-bootcamp.git
        %cd /content/rc-bootcamp/
else:
    sys.path.append(".")

from utils.interface import InteractiveViewer
from utils.interpolate import interp1d
from utils.reservoir import ESN, RidgeReadout, rls_update
from utils.style_config import plt
from utils.tester import load_from_chapter_name
from utils.tqdm import tqdm, trange
from utils.viewer import show_innate_error, show_innate_record

test_func, show_solution = load_from_chapter_name("09_internal_optimization")


# Uncomment it to see the implementations.
# import inspect
# print(inspect.getsource(Linear))
# print(inspect.getsource(RidgeReadout))
# print(inspect.getsource(ESN))
# print(inspect.getsource(rls_update))

# Or just use ??.../...?? (uncomment the following lines).
# Linear??
# RidgeReadout??
# ESN??
# rls_update??

### Implementing pre-training

Let's start by implementing the pre-training process.
As mentioned above, innate training considers the pre-synaptic connections of each node as a linear closed loop and adjusts them using FORCE learning.
The following equation shows the weight update rule based on FORCE learning for the $i$-th pre-synaptic connection ($1 \leq i \leq N$).

$$
\begin{align*}
k^{i} &= P^{i} x^{i} \\
g^{i} &= \frac{1}{1+{x^{i}}^\top k^{i}} \\
\Delta P^{i} &= g^{i}{k}^{i}{k^{i}}^\top \\
\Delta W^\mathrm{rec}_{i~\cdot} &= g^{i} (\hat{x}^{i} - {x}^{i}) {k}^{i} \\
P^{i} &\leftarrow P^{i} - \Delta P^{i} \\
W^\mathrm{rec}_{i~\cdot}  &\leftarrow W^\mathrm{rec}_{i~\cdot}  + \Delta W^\mathrm{rec}_{i~\cdot}
,\end{align*}
$$

where $P^{i} \in \mathbb{R}^{N^{i}\times N^{i}}$ is a positive definite matrix initialized as ${I}/{\alpha}$, and $\hat{x}$ is the innate trajectory.
Since we use FORCE learning based on the RLS algorithm, we can reuse `rls_update` implemented in the previous chapter.
Note that since the internal connections of the ESN are sparse (`sparse < 1.0`, $p < 1.0$), $N^{i}$ takes a smaller value than $N$ ($N^{i} \approx pN$), which reduces the computational complexity from $O(N \times N^2)$ to $O(p^2 N^3)$.

Q1.1.

Complete `InnateESN.__init__` and `InnateESN.train` by filling in the blanks in the following cell.

- `InnateESN.train`
  - Argument(s)
    - `x_target`: `np.ndarray`
      - innate trajectory $\hat{x}[k]$ ($\in \mathbb{R}^{\cdots \times N}$)
    - `x_now`: `np.ndarray`
      - $x[k]$ ($\in \mathbb{R}^{\cdots \times N}$)
  - Return(s)
    - `self.w_net`: `np.ndarray`
      - $\Delta W^\mathrm{rec}$

  - Operation(s)
    - Update `InnateESN.P` ($P^i$) with `rls_update`, which are initialized $I/\lambda$ in `__init__`.
    - Update `InnateESN.w_net` with `rls_update`.

<details><summary>tips</summary>

- [`np.nonzero`](https://numpy.org/doc/stable/reference/generated/numpy.nonzero.html)

</details>

In [ ]:
class InnateESN(ESN):
    def __init__(self, *args, lmbd=1.0, **kwargs):
        """
        Tunable ESN [Laje, R., & Buonomano, D. V. (2013). Nature neuroscience, 16(7), 925-933.]

        Args:
            alpha (float, optional): regularization parameter for RLS algorithm. Defaults to 1.0.
        """
        super(InnateESN, self).__init__(*args, **kwargs)
        self.w_pre = {}
        self.P = {}
        for post in range(self.dim):
            non_zeros = self.weight[post].nonzero()[0]
            self.w_pre[post] = non_zeros
            self.P[post] = ...  # TODO Initialize P matrix for RLS.

    def train(self, x_target, x_now=None, node_list=None):
        """
        Update the internal weight by RLS algorithm

        Args:
            x_target (np.ndarray): State(s) on an inante trajectory.
            x_now (np.ndarray, optional): Current state(s). Defaults to None (use self.x).
            node_list (list, slice, optional): Tuned nodes. Defaults to None (train all nodes).
        """
        if x_now is None:
            x_now = np.asarray(self.x)
        if node_list is None:
            node_list = range(self.dim)
        for xt, xn in zip(x_target.reshape(-1, self.dim), x_now.reshape(-1, self.dim), strict=False):
            es = xt[node_list] - xn[node_list]
            for node_id, e in zip(node_list, es, strict=False):
                x = ...  # TODO Use self.w_pre (hint: `x = xn[...]`).
                P = ...  # TODO Get P matrix for the node (hint: `P = self.P[...]`).
                g, k, P_new = ...  # TODO Use `rls_update`.
                dw = ...  # TODO Calculate dw (hint: use `g`, `e`, and `k`).
                self.P[node_id] = P_new
                self.weight[node_id, self.w_pre[node_id]] += dw[0]
        return self.weight

    def to_pickle(self, file_name):
        os.makedirs(os.path.dirname(file_name), exist_ok=True)
        P, w_pre = self.P, self.w_pre
        self.P, self.w_pre = {}, {}
        with open(file_name, mode="wb") as f:
            joblib.dump(self, f, compress=True)
        self.P, self.w_pre = P, w_pre

    @staticmethod
    def read_pickle(file_name):
        with open(file_name, mode="rb") as f:
            module = joblib.load(f)
        return module


def solution(dim, seed, x_target, x_now, node_list):
    # DO NOT CHANGE HERE.
    net = InnateESN(dim, seed=seed, node_list=node_list)
    net.train(x_target=x_target, x_now=x_now, node_list=node_list)
    return net.weight


test_func(solution, "01_01")
# show_solution("01_01", "InnateESN")  # Uncomment it to see the solution.

Q1.2.

Implement `emulate_innate` by filling in the blank in the following cell.
It will update $W^\mathrm{rec}$ using `InnateESN.train` when $k \in [t_\mathrm{b}, t_\mathrm{e}) \land k \equiv 0~ (\mathrm{mod}~t_\mathrm{every})$, and do nothing otherwise.

- `emulate_innate`
  - Argument(s):
    - `ts`: `list | np.ndarray`
    - `net`: `InnateESN`
    - `f_in`: `Callable`
      - $f^\text{in}(t)$
    - `innate_range`: `tuple(int, int)`
      - $[t_\mathrm{b}, t_\mathrm{e})$
    - `innate_node`: `list | slice`
    - `innate_every`: `int`
      - $t_\mathrm{every}$
    - `innate_func`: `Callable`
      - $\hat{x}: T \to \mathbb{R}^{\cdots \times N}$
  - Return(s):
    - `record`: `dict`
      - `'t'`: `np.ndarray`
        - $[t_0, t_1,~\ldots,~t_{T-1}]$
      - `'x'`: `np.ndarray`
        - $[x[t_0], x[t_1],~\ldots,~x[t_{T-1}]]$

In [ ]:
def emulate_innate(
    ts,
    net,
    f_in=None,
    innate_range=None,
    innate_func=None,
    innate_node=None,
    innate_every=2,
    prefix="",
    leave=True,
    display=True,
):
    record = {}
    record["t"] = np.zeros(len(ts), dtype=int)
    record["x"] = np.zeros((len(ts), *net.x.shape))
    pbar = tqdm(ts, leave=leave, display=display)
    for cnt, t in enumerate(pbar):
        pbar.set_description("{}t={:.0f}".format(prefix, t))
        u_in = np.zeros_like(net.x)
        if f_in is not None:
            u_in += f_in(t)
        net.step(u_in)
        record["t"][cnt] = t
        record["x"][cnt] = net.x
        if (innate_range is not None) and (innate_range[0] <= t < innate_range[1]):
            if cnt % innate_every == 0:
                # TODO Use `net.train` and specify `node_list=innate_node`.
                ...
    return record


def solution(ts, dim, seed, **kwargs):
    # DO NOT CHANGE HERE.
    net = InnateESN(dim, seed=seed)
    record = emulate_innate(ts, net, **kwargs)
    return record["x"]


test_func(solution, "01_02")
# show_solution("01_02", "emulate_innate")  # Uncomment it to see the solution.

Run the following cell.

- `create_target`: Generate the innate trajectory.
- `train_network`: Run the pre-training.
- `eval_network`: Evaluate the pre-trained network.
- `eval_error`: Evaluate the MSE and NRMSE between the innate trajectory and the output.

In [ ]:
def create_target(record_range, net, f_in, rnd=None):
    rnd = rnd if rnd else np.random.default_rng()
    net.x = rnd.uniform(low=-1, high=1, size=(f_in.dim, net.dim))
    record = emulate_innate(range(*record_range), net, f_in=f_in, prefix="sample ")
    return record


def train_network(record_range, net, f_in, rec_target, innate_range, innate_node, innate_every, rnd=None):
    rnd = rnd if rnd else np.random.default_rng()
    innate_func = interp1d(rec_target["x"], x=rec_target["t"], kind=1, axis=0)
    net.x = rnd.uniform(low=-1, high=1, size=(f_in.dim, net.dim))
    record = emulate_innate(
        range(*record_range),
        net,
        f_in=f_in,
        prefix="train ",
        innate_func=innate_func,
        innate_node=innate_node,
        innate_range=innate_range,
        innate_every=innate_every,
    )
    return record


def eval_network(record_range, net, f_in, eval_num=5, rnd=None):
    rnd = rnd if rnd else np.random.default_rng()
    net.x = rnd.uniform(low=-1, high=1, size=(eval_num, f_in.dim, net.dim))
    record = emulate_innate(range(*record_range), net, f_in=f_in, prefix="eval ")
    return record


def eval_error(rec_eval, rec_target, innate_range, innate_node):
    begin_id = int(innate_range[0] - rec_eval["t"][0])
    end_id = int(innate_range[1] - rec_eval["t"][0])
    diff = rec_eval["x"] - rec_target["x"][:, None]  # -> [Time_steps, Eval_num, w_pulse_num, Net_dim]
    norm = (diff[begin_id:end_id, ..., innate_node] ** 2).sum(axis=-1)  # -> [T, E, W]
    mse = norm.mean(axis=0)  # -> [E, W]

    var = (rec_target["x"][begin_id:end_id, ..., innate_node] ** 2).sum(axis=(0, 2))  # -> [W]
    nrmse = (norm.sum(axis=0) / var) ** 0.5  # -> [E, W]
    return {"mse": mse, "nrmse": nrmse}

### Run pre-training

Let's move on to the demonstration.

First, specify the experimental parameters.
Since pre-training might take a long time, the following cells create an output folder `save_dir` (`./output` by default) to allow restoring the results even after it halts.

In [ ]:
save_dir = "./result"
os.makedirs(save_dir, exist_ok=True)

seed = 1234

w_pulse_num = 1
dim, a, sr, p = 1000, 0.1, 1.6, 0.2
pulse_amp, pulse_period = 10.0, 50

alpha = 1.0
innate_epoch = 20
innate_period, innate_rate, innate_every = 3000, 0.1, 4
washout_period, record_period = 1000, 5000

rnd = np.random.default_rng(seed)
net = InnateESN(dim, a=a, sr=sr, p=p, alpha=alpha, rnd=rnd)
w_pulse = rnd.uniform(size=(w_pulse_num + 2, dim), low=-1.0, high=1.0)

innate_range = [0, innate_period]
innate_node = list(range(0, int(innate_rate * net.dim)))
record_range = [-washout_period, record_period]
pulse_range = [-pulse_period, 0]

plot_range = list(range(5))


def f_in(t):
    if -pulse_period <= t < 0:
        return pulse_amp * w_pulse[:w_pulse_num]
    else:
        return 0.0


f_in.dim = w_pulse_num

with open(f"{save_dir}/params.json", mode="w") as f:
    json.dump(
        {
            "pulse_amp": pulse_amp,
            "pulse_period": pulse_period,
            "innate_period": innate_period,
            "innate_rate": innate_rate,
            "innate_every": innate_every,
        },
        f,
        indent=4,
    )

np.save(f"{save_dir}/w_pulse.npy", w_pulse)
net.to_pickle(f"{save_dir}/net_init.pkl")

The following cell will generate the innate trajectory $\hat{x}[k]$.

In [ ]:
rec_target = create_target(record_range, net, f_in, rnd=rnd)
fig_target = show_innate_record(
    rec_target,
    plot_range,
    lw=1.5,
    color="k",
    title="innate trajectory",
    pulse_range=pulse_range,
    innate_range=innate_range,
)
np.savez_compressed(f"{save_dir}/rec_target.npz", *rec_target)
fig_target.savefig(f"{save_dir}/rec_target.png", dpi=200)

The gray area represents the period when the pulse input is given.
The pink area shows the pre-training period $[t_\mathrm{b}, t_\mathrm{e})$.

In [ ]:
def eval_network(record_range, net, f_in, eval_num=5, rnd=None):  # noqa: F811
    rnd = rnd if rnd else np.random.default_rng()
    net.x = rnd.uniform(low=-1, high=1, size=(eval_num, f_in.dim, net.dim))
    record = emulate_innate(range(*record_range), net, f_in=f_in, prefix="eval ")
    return record


rec_init = eval_network(record_range, net, f_in, rnd=rnd)
fig = show_innate_record(
    rec_target,
    plot_range,
    lw=1.5,
    ls=":",
    clear=True,
    color="k",
    pulse_range=pulse_range,
    innate_range=innate_range,
)
fig = show_innate_record(
    rec_init,
    plot_range,
    lw=0.5,
    ls="-",
    fig=fig,
    cmap=plt.get_cmap("tab10"),
    title="eval (before pre-training)",
)
fig.savefig(f"{save_dir}/eval/init.png", dpi=200)

This figure shows that the ESN cannot reproducibly generate the innate trajectories (dotted lines) at this point.

The following cell runs the pre-training iteratively.
This will take 20-30 minutes, so please wait until it finishes.

In [ ]:
record = {}
best_score = np.array(np.inf)
figs = defaultdict(lambda: None)

pbar = trange(innate_epoch)
out_tqdm, out_figure = Output(), Output()
display(out_tqdm)
display(out_figure)
for epoch in pbar:
    pbar.set_description("best:{:.2e}".format(best_score))
    with out_tqdm:
        clear_output(wait=True)
        # Training phase
        rec_train = train_network(
            record_range,
            net,
            f_in,
            rec_target,
            innate_range,
            innate_node,
            innate_every,
            rnd=rnd,
        )
        figs["train"] = show_innate_record(
            rec_target,
            plot_range,
            lw=1.5,
            ls=":",
            clear=True,
            color="black",
            fig=figs["train"],
            pulse_range=pulse_range,
            innate_range=innate_range,
        )
        figs["train"] = show_innate_record(
            rec_train,
            plot_range,
            lw=1.0,
            ls="-",
            fig=figs["train"],
            cmap=plt.get_cmap("tab10"),
            title=f"train (epoch #{epoch})",
        )
        figs["train"].savefig("{}/train/{:03d}.png".format(save_dir, epoch), dpi=200)

        # Evaluation phase
        rec_eval = eval_network(record_range, net, f_in, rnd=rnd)
        figs["eval"] = show_innate_record(
            rec_target,
            plot_range,
            lw=1.5,
            ls=":",
            clear=True,
            color="black",
            fig=figs["eval"],
            pulse_range=pulse_range,
            innate_range=innate_range,
        )
        figs["eval"] = show_innate_record(
            rec_eval,
            plot_range,
            lw=0.5,
            ls="-",
            fig=figs["eval"],
            cmap=plt.get_cmap("tab10"),
            title=f"eval (epoch #{epoch})",
        )
        figs["eval"].savefig("{}/eval/{:03d}.png".format(save_dir, epoch), dpi=200)

        # Record evaluation error
        rec = eval_error(rec_eval, rec_target, innate_range, innate_node)
        best_score = min(best_score, rec["nrmse"].sum())
        rec["best"] = best_score

        for key, val in rec.items():
            if key not in record:
                record[key] = np.zeros((innate_epoch, *val.shape))
            record[key][epoch] = val

    with out_figure:
        clear_output(wait=True)
        figs["nrmse"] = show_innate_error(record["nrmse"][: epoch + 1], fig=figs["nrmse"])
        figs["nrmse"].savefig(f"{save_dir}/nrmse.png", dpi=200)
        for _name, fig in figs.items():
            size = fig.get_size_inches()
            fig.set_size_inches(8, 3)
            display(fig)
            fig.set_size_inches(size)

net.to_pickle(f"{save_dir}/net_term.pkl")
np.savez_compressed(f"{save_dir}/record.npz", **record)

for _name, fig in figs.items():
    fig.close()

You will see that the ESN gradually becomes able to reproducibly generate complex high-dimensional chaos despite different initial values.
Check the figures saved in `{save_dir}/eval`.

Q1.3. (Advanced)

- Vary the number of nodes and observe the differences in MSE.
- Try a longer `innate_period` and compare the pre-training performance.
- Increase `w_pulse_num` and pre-train with multiple inputs.

### Implementing post-training

Let's train the readout layer using a pre-trained ESN.
The following cell loads the ESN saved in `{load_dir}/net_term.pkl` and the parameter file saved in `{load_dir}/params.json`.
You can skip the previous cell and start from here if you have already completed the pre-training.

In [ ]:
load_dir = "./result"

net_init = InnateESN.read_pickle(f"{load_dir}/net_init.pkl")
net_term = InnateESN.read_pickle(f"{load_dir}/net_term.pkl")
w_pulse = np.load(f"{load_dir}/w_pulse.npy")
with open(f"{load_dir}/params.json", mode="r") as f:
    params = json.load(f)

pulse_amp = params["pulse_amp"]
pulse_period = params["pulse_period"]
innate_period = params["innate_period"]

The following cell compares the trajectories of the ESN before and after pre-training.

In [ ]:
seed = 1234
w_pulse_num = 1
washout_period, record_period = 1000, 10000
record_range = [-washout_period, record_period]
pulse_range = [-pulse_period, 0]
innate_range = [0, innate_period]
plot_range = list(range(5))

rnd = np.random.default_rng(seed)


def f_in(t):
    if -pulse_period <= t < 0:
        return pulse_amp * w_pulse[:w_pulse_num]
    else:
        return 0.0


f_in.dim = w_pulse_num

rec_sample_init = eval_network(record_range, net_init, f_in, rnd=rnd)
fig = show_innate_record(
    rec_sample_init,
    plot_range,
    lw=1.0,
    title="initial",
    pulse_range=pulse_range,
    innate_range=innate_range,
)

rec_sample_term = eval_network(record_range, net_term, f_in, rnd=rnd)
fig = show_innate_record(
    rec_sample_term,
    plot_range,
    lw=1.0,
    title="pre-trained",
    pulse_range=pulse_range,
    innate_range=innate_range,
)

Next, let's prepare the target output time series for the readout layer.
`abc.csv` and `star.csv` are available in `./data/09_internal_optimization`.
Let's use `abc.csv` this time.

In [ ]:
data = np.loadtxt("./data/09_internal_optimization/abc.csv", delimiter=",")

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(data[:, 0], data[:, 1])
ax.set_aspect("equal")

The pre-trained ESN reproducibly generates innate trajectories for $t\in[0, t_\mathrm{innate})$ after the input is given.
The following cell trains the readout layer to map this high-dimensional chaos to the prepared time series.

In [ ]:
ts, xs = rec_sample_term["t"], rec_sample_term["x"]

x_train = xs[washout_period : washout_period + innate_period, :, 0, :]

target_func = interp1d(data, kind=1, axis=0)
ds = target_func(np.linspace(0, 1, innate_period))
ds = np.broadcast_to(ds[:, None, :], (*x_train.shape[:-1], ds.shape[-1]))

w_out = RidgeReadout(net_term.dim, 2, lmbd=1e-2)
weight, bias = w_out.train(x_train.reshape(-1, x_train.shape[-1]), ds.reshape(-1, ds.shape[-1]))

`InteractiveView` is an interface where you can interactively give inputs.
Run the following cell and check that the target trajectory `abc.csv` is properly output after the input is given.

In [ ]:
%matplotlib widget

import copy

try:
    del viewer  # type: ignore
except NameError:
    pass

net_term_cp = copy.deepcopy(net_term)

net_term_cp.x = rnd.uniform(-1, 1, net_term_cp.dim)
viewer = InteractiveViewer(
    net_term_cp,
    w_out,
    w_pulse,
    pulse_amp,
    pulse_period,
    plot_num=5,
    input_num=w_pulse_num,
    max_time_steps=10000,
    cmap="Greens",
)

viewer.view()

Q2.1. (Advanced)

- By increasing the `input_num` argument in `InteractiveView`, you can increase the number of input types. However, the network has not been trained for these inputs yet, so only meaningless outputs will be obtained. Increase the number of inputs to 2 (`w_pulse_num=2`) and perform both pre-training and post-training so that a different input generates a "star" output (`star.csv`) (this can be done with minor modifications).

Q2.2. (Advanced)

- Full FORCE can be implemented with almost no code changes from innate training. Implement `FullFORCEESN` inheriting from `InnateESN` by referring to [2].

## References

[1] Laje, R., & Buonomano, D. V. (2013). *Robust timing and motor patterns by taming chaos in recurrent neural networks*. Nature Neuroscience, 16(7), 925–933. https://doi.org/10.1038/nn.3405

[2] DePasquale, B., Cueva, C. J., Rajan, K., Escola, G. S., & Abbott, L. F. (2018). *full-FORCE: A target-based method for training recurrent networks*. PLOS ONE, 13(2), e0191527. https://doi.org/10.1371/journal.pone.0191527

[3] Sussillo, D., & Abbott, L. F. (2009). *Generating Coherent Patterns of Activity from Chaotic Neural Networks*. Neuron, 63(4), 544–557. https://doi.org/10.1016/j.neuron.2009.07.018

[4] Inoue, K., Nakajima, K., & Kuniyoshi, Y. (2020). *Designing spontaneous behavioral switching via chaotic itinerancy*. Science Advances, 6(46), eabb3989. https://doi.org/10.1126/sciadv.abb3989